In [1]:
import numpy as np
import pandas as pd
import datetime
import sklearn
from sklearn.metrics import *
from sklearn import preprocessing
import itertools

In [2]:
def extend_to_right_boundary(df, max_time):
  df_tmp = df.reset_index(level = 0, drop = True).sort_index(ascending=True)
  min_left = min(df.index)[1]
  extended_time = pd.date_range(start = min_left, end = max_time, freq ='W-Mon')
  df_tmp = df_tmp.reindex(extended_time, fill_value = 0.0)
  df_tmp.index.name = "DateTime"
  #print((df.index.get_level_values(level = 0).unique(), extended_time))

  # this does not work
  # weekly_sales.loc[[3129],:].reindex(index = multiindex.at[3129], level = "DateTime", fill_value = 0.0)

  return df_tmp

In [3]:
def prepare_sales_data(sales_df, categories_df, traffic_df, extend_to_common_horizon = False, extend_only_to_max_date = False, shift_list = [None]):
  sales_df_copy = sales_df.copy()
  sales_df_copy['DateTime'] = sales_df_copy['week_starting_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
  weekly_sales = sales_df_copy.groupby(["product_id", "DateTime"], as_index=True).agg({'sales':'sum'}) # czy to jest potrzebne?

  if extend_to_common_horizon:
    if extend_only_to_max_date:
      max_time_value = max(weekly_sales.index.get_level_values(level = 1))
      expanded_full = weekly_sales.groupby(level=0).apply(lambda x: extend_to_right_boundary(x, max_time_value))
    else:
      t1 = weekly_sales.index.get_level_values(level = 0).unique()
      t2 = weekly_sales.index.get_level_values(level = 1).unique()
      new_index = list(itertools.product(t1, t2))
      expanded_full = weekly_sales.reindex(pd.MultiIndex.from_tuples(new_index, names=['product_id', 'DateTime']), fill_value=0)
  else:
    expanded_full = weekly_sales.groupby(level=0).apply(lambda x: x.reset_index(level=0, drop=True).asfreq("W-Mon", fill_value = 0.0))
    #expanded_full = weekly_sales.groupby(level=0).apply(lambda x: x.reset_index(level=0, drop=True).asfreq("W-Mon").fillna(0))
  
  

  full_sales_dataset = expanded_full\
  .merge(categories_df, left_index=True, right_index = True,  how = 'left')\
  .merge(traffic_df, left_index = True,  right_index = True, how = 'left')

  full_sales_dataset["Date"] = full_sales_dataset.index.get_level_values(level = 1)
  
  # adding columns based on start time
  full_sales_dataset['Week_numb'] = full_sales_dataset['Date'].apply(lambda x: int(x.strftime("%V")))
  full_sales_dataset['YW'] = full_sales_dataset['Date'].apply(lambda x: int(x.strftime("%Y%V")))
  full_sales_dataset['Month'] = full_sales_dataset['Date'].apply(lambda x: int(x.month))

  if shift_list[0] is not None:
    for i in shift_list:
      # do this but within groups, otherwise there is a bug
      full_sales_dataset[[f"sales_lag_{i}W", f"traffic_lag_{i}W"]] = full_sales_dataset.groupby(level = 0)[['sales', 'traffic']].shift(i, fill_value = 0.0)

  #full_sales_dataset["sales_to_traffic"] = full_sales_dataset.sales/full_sales_dataset.traffic
  
  full_sales_dataset.drop("Date", axis = 1, inplace = True)


  return full_sales_dataset
